In [73]:
from shapely.geometry import Point, Polygon
from typing import NamedTuple
import pathlib
import shapefile
import pandas as pd
import csv
import random
import geopandas as gpd

In [74]:
class NERC(NamedTuple):
    id: str
    polygon: Polygon

class Facility(NamedTuple):
    entityid: int
    plantid: int
    entityname: str
    plantname: str
    capacity: int
    latitude: float
    longitude: float
    tech: str

In [100]:
nerc_file = "../data/nerc_regions/NERC_Regions_EIA"
generator_file = "../data/Renewables/july_generator2023.xlsx"

In [101]:
def load_shapefiles(path: pathlib.Path) -> NERC:
    """
    Extract and parse polygons from NERC shapefiles.
    """
    nercs = []
    with shapefile.Reader(path) as sf:
        for shape_rec in sf.shapeRecords():
            nercs.append(
                NERC(
                    id=shape_rec.record["NERC"],
                    polygon=Polygon(shape_rec.shape.points),
                )
            )
    return nercs

In [102]:
nerc = load_shapefiles(nerc_file)

In [78]:
def load_frs_csv(path: pathlib.Path) -> list[Facility]:
    """
    Given a CSV containing facility data, return a list of Facility objects.
    """
    facilities = []
    df = pd.read_excel(generator_file, sheet_name="Operating", skiprows=2)
    df = df.reset_index() 
    for index, row in df.iterrows():
            facilities.append(
                Facility(
                    entityid=row["Entity ID"],
                    entityname=row["Entity Name"],
                    plantid=row["Plant ID"],
                    plantname=row["Plant Name"],
                    capacity=row["Nameplate Capacity (MW)"],
                    latitude=row["Latitude"],
                    longitude=row["Longitude"],
                    tech=row["Technology"]
                )
            )
    return facilities

In [79]:
generator = load_frs_csv(generator_file)

In [80]:
def spatial_join(facilities: list, nerc: list) -> list[tuple[str, str]]:

    facility_nerc = []

    for facility in facilities: 
        facility_point = Point(facility.longitude, facility.latitude)
        for id, poly in nerc:
            if poly.contains(facility_point):
                facility_nerc.append((facility.id, id))
                break
            else:
                continue
    return facility_nerc

In [103]:
facility_by_nerc = spatial_join(generator, nerc)

In [82]:
# debugging
facility = generator[10]
facility_point = Point(facility.longitude, facility.latitude)
print(facility_point)

serc_region = nerc[3]
print(serc_region)
id, poly = serc_region

if poly.contains(facility_point):
    print("Contained")
else:
    print("Not Contained")

POINT (-88.0103 31.0069)
NERC(id='SERC', polygon=<POLYGON ((-9123196.218 2816863.693, -9123190.273 2817148.175, -9123352.49 2...>)
Not Contained
